# Preparing data for Power Bi Dashboard

In [262]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [263]:
departures = pd.read_csv('../../data/Departure.csv',encoding='latin1',low_memory=False)

In [264]:
arrivals = pd.read_csv('../../data/Arrivals.csv', encoding='utf-8-sig', low_memory=False)

In [265]:
cancellations = pd.read_csv('../../data/Airlines_Cancellation.csv',encoding='utf-8-sig',low_memory=False)

In [266]:
departures.shape

(497696, 16)

In [267]:
arrivals.shape

(513001, 16)

In [268]:
cancellations.shape

(10370, 6)

In [272]:
departures.rename(columns={'Date (MM/DD/YYYY)':'Flight Date'},inplace =True)

In [274]:
arrivals.rename(columns={'Date (MM/DD/YYYY)':'Flight Date'},inplace =True)

In [280]:
cancellations.rename(columns={'Date (MM/DD/YYYY)':'Flight Date'},inplace =True)

In [281]:
departures['Flight Date'] = pd.to_datetime(departures['Flight Date'])

In [282]:
arrivals['Flight Date'] = pd.to_datetime(arrivals['Flight Date'])

In [284]:
cancellations['Flight Date'] = pd.to_datetime(cancellations['Flight Date'])

In [285]:
departures = departures[departures['Flight Date'].dt.year <= 2024]

In [286]:
arrivals = arrivals[arrivals['Flight Date'].dt.year <= 2024]

In [287]:
cancellations = cancellations[cancellations['Flight Date'].dt.year <= 2024]

In [288]:
### Converting Date columns to datetime
departures['Flight Date'] = pd.to_datetime(departures['Flight Date'])
arrivals['Flight Date'] = pd.to_datetime(arrivals['Flight Date'])  # adjust if your column name is 'Date'
cancellations['Flight Date'] = pd.to_datetime(cancellations['Flight Date'])

In [289]:
### Function to classify COVID Phase
def covid_phase(date):
    if date < pd.Timestamp('2020-01-01'):
        return 'Pre-COVID'
    elif date <= pd.Timestamp('2021-12-31'):
        return 'During-COVID'
    else:
        return 'Post-COVID'

In [290]:
### Applying the covid_phase function to each dataframe
departures['COVID Phase'] = departures['Flight Date'].apply(covid_phase)
arrivals['COVID Phase'] = arrivals['Flight Date'].apply(covid_phase)
cancellations['COVID Phase'] = cancellations['Flight Date'].apply(covid_phase)

In [291]:
departures

,Carrier Code,Airlines,Flight Date,Flight Number,Destination Airport,Destination City,Scheduled departure time,Actual departure time,Scheduled elapsed time (Minutes),Actual elapsed time (Minutes),Departure delay (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),COVID Phase
0,AA,American Airlines,2018-01-01,469,PHL,"Philadelphia, PA",9:24:00 AM,9:35:00 AM,124,109,11,0,0,0,0,0,Pre-COVID
1,AA,American Airlines,2018-01-01,602,DFW,"DallasFort Worth, TX",8:51:00 PM,9:13:00 PM,139,132,22,0,0,0,0,15,Pre-COVID
2,AA,American Airlines,2018-01-01,820,PHL,"Philadelphia, PA",7:00:00 AM,6:59:00 AM,134,124,-1,0,0,0,0,0,Pre-COVID
3,AA,American Airlines,2018-01-01,1065,DFW,"DallasFort Worth, TX",9:28:00 AM,9:49:00 AM,136,127,21,0,0,0,0,0,Pre-COVID
4,AA,American Airlines,2018-01-01,1439,DFW,"DallasFort Worth, TX",3:20:00 PM,3:21:00 PM,134,119,1,0,0,0,0,0,Pre-COVID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
497691,WN,Southwest Airlines,2024-12-31,4966,PHX,"Phoenix, AZ",5:50:00 AM,5:49:00 AM,240,258,-1,0,0,17,0,0,Post-COVID
497692,WN,Southwest Airlines,2024-12-31,5185,MCI,"Kansas City, MO",8:35:00 AM,8:44:00 AM,105,100,9,0,0,0,0,0,Post-COVID
497693,WN,Southwest Airlines,2024-12-31,5253,DEN,"Denver, CO",2:50:00 PM,2:47:00 PM,180,186,-3,0,0,0,0,0,Post-COVID
497694,WN,Southwest Airlines,2024-12-31,5302,AUS,"Austin, TX",8:50:00 AM,8:54:00 AM,145,144,4,0,0,0,0,0,Post-COVID


In [292]:
dep_agg = departures.groupby(['Airlines','Flight Date','COVID Phase']).agg({
    'Departure delay (Minutes)':'mean',
    'Delay Carrier (Minutes)':'sum',
    'Delay Weather (Minutes)':'sum',
    'Delay National Aviation System (Minutes)':'sum',
    'Delay Security (Minutes)':'sum',
    'Delay Late Aircraft Arrival (Minutes)':'sum',
    'Flight Number':'count'   # Total depatures flights
}).reset_index().rename(columns={'Flight Number':'Total Flights'})

In [293]:
arrivals

,Carrier Code,Airlines,Flight Date,Flight Number,Origin Airport,Origin City,Scheduled Arrival Time,Actual Arrival Time,Scheduled Elapsed Time (Minutes),Actual Elapsed Time (Minutes),Arrival Delay (Minutes),Delay Carrier (Minutes),Delay Weather (Minutes),Delay National Aviation System (Minutes),Delay Security (Minutes),Delay Late Aircraft Arrival (Minutes),COVID Phase
0,AA,American Airlines,2018-01-01,829,PHL,"Philadelphia, PA",9:34:00 PM,9:29:00 PM,139,129,-5,0,0,0,0,0,Pre-COVID
1,AA,American Airlines,2018-01-01,851,CLT,"Charlotte, NC",12:09:00 PM,12:05:00 PM,89,90,-4,0,0,0,0,0,Pre-COVID
2,AA,American Airlines,2018-01-01,1396,DFW,"Dallas–Fort Worth, TX",8:14:00 PM,8:09:00 PM,108,98,-5,0,0,0,0,0,Pre-COVID
3,AA,American Airlines,2018-01-01,1439,DFW,"Dallas–Fort Worth, TX",2:32:00 PM,2:24:00 PM,107,104,-8,0,0,0,0,0,Pre-COVID
4,AA,American Airlines,2018-01-01,1463,LAX,"Los Angeles, CA",9:21:00 PM,10:27:00 PM,231,224,66,13,0,0,0,53,Pre-COVID
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
512996,WN,Southwest Airlines,2024-12-31,4820,TPA,"Tampa, FL",9:05:00 AM,9:08:00 AM,115,114,3,0,0,0,0,0,Post-COVID
512997,WN,Southwest Airlines,2024-12-31,4878,DEN,"Denver, CO",7:40:00 PM,7:25:00 PM,145,131,-15,0,0,0,0,0,Post-COVID
512998,WN,Southwest Airlines,2024-12-31,4960,MKE,"Milwaukee, WI",5:00:00 PM,4:53:00 PM,90,85,-7,0,0,0,0,0,Post-COVID
512999,WN,Southwest Airlines,2024-12-31,5036,BUR,"Burbank, CA",1:10:00 PM,12:26:00 PM,240,201,-44,0,0,0,0,0,Post-COVID


In [294]:
arr_agg = arrivals.groupby(['Airlines','Flight Date','COVID Phase']).agg({
    'Arrival Delay (Minutes)':'mean',
    'Delay Carrier (Minutes)':'sum',
    'Delay Weather (Minutes)':'sum',
    'Delay National Aviation System (Minutes)':'sum',
    'Delay Security (Minutes)':'sum',
    'Delay Late Aircraft Arrival (Minutes)':'sum',
    'Flight Number':'count' ## Total Arrivals
}).reset_index().rename(columns={'Flight Number':'Total Flights'})

In [295]:
cancel_agg = cancellations.groupby(['Airlines','Flight Date','COVID Phase']).agg({
    'Flight_Number':'count'   # Number of cancelled flights
}).reset_index().rename(columns={'Flight_Number':'Cancelled Flights'})

In [296]:
cancellations

,Carrier Code,Airlines,Flight Date,Flight_Number,Destination Airport,Destination City,COVID Phase
0,AA,American Airlines,2018-01-04,469,PHL,"Philadelphia, PA",Pre-COVID
1,AA,American Airlines,2018-01-08,1899,PHL,"Philadelphia, PA",Pre-COVID
2,AA,American Airlines,2018-01-11,1439,DFW,"Dallas–Fort Worth, TX",Pre-COVID
3,AA,American Airlines,2018-01-17,1694,CLT,"Charlotte, NC",Pre-COVID
4,AA,American Airlines,2018-01-17,1736,CLT,"Charlotte, NC",Pre-COVID
...,...,...,...,...,...,...,...
10013,WN,Southwest Airlines,2024-12-20,3034,SAN,"San Diego, CA",Post-COVID
10014,WN,Southwest Airlines,2024-12-20,3239,BOS,"Boston, MA",Post-COVID
10015,WN,Southwest Airlines,2024-12-28,2435,DAL,"Dallas, TX",Post-COVID
10016,WN,Southwest Airlines,2024-12-29,6310,TPA,"Tampa, FL",Post-COVID


In [297]:
dep_cancel = dep_agg.merge(cancel_agg, on=['Airlines','Flight Date','COVID Phase'], how='left')
dep_cancel['Cancelled Flights'] = dep_cancel['Cancelled Flights'].fillna(0)
dep_cancel['Cancellation Rate (%)'] = dep_cancel['Cancelled Flights'] / dep_cancel['Total Flights'] * 100

In [298]:
merged = dep_cancel.merge(arr_agg, on=['Airlines','Flight Date','COVID Phase'], how='left', suffixes=('_Dep','_Arr'))

In [299]:
merged

,Airlines,Flight Date,COVID Phase,Departure delay (Minutes),Delay Carrier (Minutes)_Dep,Delay Weather (Minutes)_Dep,Delay National Aviation System (Minutes)_Dep,Delay Security (Minutes)_Dep,Delay Late Aircraft Arrival (Minutes)_Dep,Total Flights_Dep,Cancelled Flights,Cancellation Rate (%),Arrival Delay (Minutes),Delay Carrier (Minutes)_Arr,Delay Weather (Minutes)_Arr,Delay National Aviation System (Minutes)_Arr,Delay Security (Minutes)_Arr,Delay Late Aircraft Arrival (Minutes)_Arr,Total Flights_Arr
0,Alaska Airlines,2018-01-01,Pre-COVID,-1.000000,0,0,0,0,0,1,0.0,0.000000,-21.000000,0.0,0.0,0.0,0.0,0.0,1.0
1,Alaska Airlines,2018-01-02,Pre-COVID,-4.000000,0,0,0,0,0,1,0.0,0.000000,-1.000000,0.0,0.0,0.0,0.0,0.0,1.0
2,Alaska Airlines,2018-01-03,Pre-COVID,-2.000000,0,0,0,0,0,1,0.0,0.000000,-16.000000,0.0,0.0,0.0,0.0,0.0,1.0
3,Alaska Airlines,2018-01-04,Pre-COVID,1.000000,0,0,0,0,0,1,0.0,0.000000,-1.000000,0.0,0.0,0.0,0.0,0.0,1.0
4,Alaska Airlines,2018-01-05,Pre-COVID,5.000000,0,0,0,0,0,1,0.0,0.000000,-33.000000,0.0,0.0,0.0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19320,United Airlines,2024-12-27,Post-COVID,47.200000,392,0,20,0,200,15,0.0,0.000000,16.933333,72.0,0.0,15.0,0.0,216.0,15.0
19321,United Airlines,2024-12-28,Post-COVID,0.000000,0,0,207,0,0,12,0.0,0.000000,17.300000,129.0,0.0,18.0,0.0,88.0,10.0
19322,United Airlines,2024-12-29,Post-COVID,30.846154,81,0,126,0,263,13,1.0,7.692308,26.533333,130.0,0.0,2.0,0.0,339.0,15.0
19323,United Airlines,2024-12-30,Post-COVID,5.733333,85,0,4,0,0,15,0.0,0.000000,-13.666667,15.0,0.0,0.0,0.0,9.0,15.0


In [300]:
merged.to_csv('Flight_Delay_merged_powerbi.csv', index=False)

In [301]:
merged['Total Flights_Dep'].sum()

np.int64(448284)

In [302]:
arrival_city_agg = arrivals.groupby(['Origin City','Airlines','COVID Phase']).agg({'Arrival Delay (Minutes)': 'mean','Flight Number': 'count'}).reset_index()

In [303]:
arrival_city_agg.sort_values(by='Arrival Delay (Minutes)',ascending=False)


,Origin City,Airlines,COVID Phase,Arrival Delay (Minutes),Flight Number
6,"Austin, TX",American Airlines,Post-COVID,55.947368,19
289,"Trenton, NJ",Frontier Airlines,Pre-COVID,30.724771,218
145,"Madison, WI",Delta Airlines,Pre-COVID,28.000000,2
132,"Long Island, NY",Southwest Airlines,During-COVID,25.901408,213
48,"Chicago, IL",Frontier Airlines,Post-COVID,23.040984,122
...,...,...,...,...,...
225,"Phoenix, AZ",American Airlines,During-COVID,-11.372032,379
27,"Bozeman, MT",Southwest Airlines,Post-COVID,-14.222222,9
28,"Buffalo, NY",Southwest Airlines,During-COVID,-14.952381,21
237,"Raleigh-Durham, NC",Delta Airlines,Pre-COVID,-18.000000,1


In [304]:
arrival_city_agg.rename(columns={'Arrival Delay (Minutes)': 'Avg Arrival Delay (min)','Flight Number': 'Total Flights'}, inplace=True)

In [305]:
arrival_city_agg

,Origin City,Airlines,COVID Phase,Avg Arrival Delay (min),Total Flights
0,"Atlanta, GA",Delta Airlines,During-COVID,-1.663963,4809
1,"Atlanta, GA",Delta Airlines,Post-COVID,4.249364,9035
2,"Atlanta, GA",Delta Airlines,Pre-COVID,1.289241,7129
3,"Atlanta, GA",Southwest Airlines,During-COVID,-0.992244,1805
4,"Atlanta, GA",Southwest Airlines,Post-COVID,2.492707,2468
...,...,...,...,...,...
297,"Washington, DC",Southwest Airlines,Pre-COVID,0.458861,2528
298,"Washington, DC",United Airlines,During-COVID,-3.653543,127
299,"Washington, DC",United Airlines,Post-COVID,7.434765,2491
300,"Washington, DC",United Airlines,Pre-COVID,9.439060,681


In [306]:
departure_city_agg = departures.groupby(['Destination City','Airlines','COVID Phase']).agg({'Departure delay (Minutes)': 'mean','Flight Number': 'count'}).reset_index()

In [307]:
departure_city_agg.rename(columns={'Departure delay (Minutes)': 'Avg Departure Delay (min)','Flight Number': 'Total Flights'}, inplace=True)

In [308]:
departure_city_agg

,Destination City,Airlines,COVID Phase,Avg Departure Delay (min),Total Flights
0,"Atlanta, GA",Delta Airlines,During-COVID,0.146478,4813
1,"Atlanta, GA",Delta Airlines,Post-COVID,5.921178,9033
2,"Atlanta, GA",Delta Airlines,Pre-COVID,4.740710,7131
3,"Atlanta, GA",Southwest Airlines,During-COVID,6.285556,1800
4,"Atlanta, GA",Southwest Airlines,Post-COVID,9.311871,2485
...,...,...,...,...,...
299,"Washington, DC",Southwest Airlines,Pre-COVID,10.063687,2528
300,"Washington, DC",United Airlines,During-COVID,-1.566372,113
301,"Washington, DC",United Airlines,Post-COVID,9.680396,2525
302,"Washington, DC",United Airlines,Pre-COVID,15.658789,677


In [309]:
cancellations.columns = cancellations.columns.str.strip()

In [310]:
cancel_city_agg = cancellations.groupby(['Destination City','Airlines', 'COVID Phase']).agg({'Flight_Number': 'count'}).reset_index()

In [311]:
cancellations.columns

Index(['Carrier Code', 'Airlines', 'Flight Date', 'Flight_Number',
       'Destination Airport', 'Destination City', 'COVID Phase'],
      dtype='object')

In [312]:
# Remove trailing/leading spaces and periods
arrival_city_agg['Origin City'] = arrival_city_agg['Origin City'].str.strip().str.replace('.', '', regex=False)
departure_city_agg['Destination City'] = departure_city_agg['Destination City'].str.strip().str.replace('.', '', regex=False)
cancel_city_agg['Destination City'] = cancel_city_agg['Destination City'].str.strip().str.replace('.', '', regex=False)

In [313]:
arrival_city_agg.to_csv('arrival_city_agg.csv', index=False)

In [314]:
departure_city_agg.to_csv('departure_city_agg.csv', index=False)

In [315]:
cancel_city_agg.to_csv('cancel_city_agg.csv', index=False)